<a href="https://colab.research.google.com/github/Nboev2004/InClassAssignments/blob/main/MAS_342_hw_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Question 1, part d

import networkx as nx

# Create an undirected weighted graph
G = nx.Graph()

# Add all offices and distances (edges with weights)
edges = [
    ("Main", "B1", 190),
    ("Main", "B2", 70),
    ("Main", "B3", 115),
    ("Main", "B4", 270),
    ("Main", "B5", 160),
    ("B1", "B2", 100),
    ("B1", "B3", 110),
    ("B1", "B4", 215),
    ("B1", "B5", 50),
    ("B2", "B3", 140),
    ("B2", "B4", 120),
    ("B2", "B5", 220),
    ("B3", "B4", 175),
    ("B3", "B5", 80),
    ("B4", "B5", 310),
]

G.add_weighted_edges_from(edges)

# Compute the minimum spanning tree using Kruskal's algorithm
T = nx.minimum_spanning_tree(G, algorithm="kruskal")

print("Edges in the MST:")
total_length = 0
for u, v, data in T.edges(data=True):
    w = data["weight"]
    print(f"{u} – {v}: {w}")
    total_length += w

print("Total cable length in MST:", total_length, "miles")


Edges in the MST:
Main – B2: 70
B1 – B5: 50
B1 – B2: 100
B2 – B4: 120
B3 – B5: 80
Total cable length in MST: 420 miles


In [8]:
# Question 2, part b

!pip install gurobipy
import gurobipy as gp
from gurobipy import GRB

# Sets
nodes = ["A", "B", "C", "D", "E", "F", "G"]

arcs = {
    ("A", "B"): 100,
    ("A", "C"): 100,
    ("B", "C"): 30,
    ("B", "D"): 80,
    ("B", "E"): 60,
    ("C", "E"): 40,
    ("C", "F"): 50,
    ("D", "E"): 30,
    ("D", "G"): 70,
    ("E", "F"): 20,
    ("E", "G"): 100,
    ("F", "G"): 80,
}

source = "A"
sink = "G"

# Model
m = gp.Model("max_flow")

# Decision variables: flow on each arc
flow = m.addVars(arcs.keys(), lb=0, ub=[arcs[a] for a in arcs], name="f")

# Objective: maximize total flow out of source (or into sink)
m.setObjective(gp.quicksum(flow[source, j] for j in nodes if (source, j) in arcs),
               GRB.MAXIMIZE)

# Flow conservation constraints
for k in nodes:
    if k == source or k == sink:
        continue
    inflow  = gp.quicksum(flow[i, k] for i in nodes if (i, k) in arcs)
    outflow = gp.quicksum(flow[k, j] for j in nodes if (k, j) in arcs)
    m.addConstr(inflow == outflow, name=f"flow_bal_{k}")

# Source = sink consistency (optional but nice)
in_sink  = gp.quicksum(flow[i, sink] for i in nodes if (i, sink) in arcs)
out_src  = gp.quicksum(flow[source, j] for j in nodes if (source, j) in arcs)
m.addConstr(in_sink == out_src, name="total_flow")

# Optimize
m.optimize()

print("\nOptimal objective (max flow):", m.objVal)

print("\nArc flows:")
for (i, j) in arcs:
    if flow[i, j].X > 1e-6:
        print(f"{i} -> {j}: {flow[i, j].X}")


Restricted license - for non-production use only - expires 2027-11-29
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 12 columns and 24 nonzeros (Max)
Model fingerprint: 0x4f7ebad5
Model has 2 linear objective coefficients
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+01, 1e+02]
  RHS range        [0e+00, 0e+00]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 5 rows, 11 columns, 22 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9009100e+02   2.100910e+02   0.000000e+00      0s
       4    1.9000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.900000000e+02

Optimal ob

In [4]:
# Question 2, part c

import networkx as nx

G = nx.DiGraph()

# Add edges with capacities
G.add_edge("A", "B", capacity=100)
G.add_edge("A", "C", capacity=100)
G.add_edge("B", "C", capacity=30)
G.add_edge("B", "D", capacity=80)
G.add_edge("B", "E", capacity=60)
G.add_edge("C", "E", capacity=40)
G.add_edge("C", "F", capacity=50)
G.add_edge("D", "E", capacity=30)
G.add_edge("D", "G", capacity=70)
G.add_edge("E", "F", capacity=20)
G.add_edge("E", "G", capacity=100)
G.add_edge("F", "G", capacity=80)

# Compute maximum flow from A to G
flow_value, flow_dict = nx.maximum_flow(G, "A", "G", capacity="capacity")

print("Maximum flow from A to G:", flow_value)
print("\nFlow on each arc:")
for u in flow_dict:
    for v, f in flow_dict[u].items():
        if f > 0:
            print(f"{u} -> {v}: {f}")

Maximum flow from A to G: 190

Flow on each arc:
A -> B: 100
A -> C: 90
B -> D: 80
B -> E: 20
C -> E: 40
C -> F: 50
D -> E: 10
D -> G: 70
E -> G: 70
F -> G: 50
